In [1]:
import pandas as pd

import requests
import json
from unidecode import unidecode

from kiblib.utils.db import DbConn

In [2]:
def mod_borrower(userid, data):
    api_url = "http://cataloguekoha.ntrbx.local/cgi-bin/koha/rest.pl/user"
    url = f"{api_url}/{userid}"
    data = json.dumps(data)
    data2mod = f"data={data}"
    #print(data2mod)
    response = requests.put(url, data=data2mod)
    #print(f"{userid} : {response.content}")

In [3]:
db_conn = DbConn().create_engine()

# Préparation des données

In [28]:
adherents = pd.read_json("http://cataloguekoha.ntrbx.local/cgi-bin/koha/rest.pl/user")
adherents = adherents[['borrowernumber', 'cardnumber', 'userid', 'address', 'address2',
                       'city', 'state', 'zipcode', 'altcontactcountry',
                       'categorycode', 'dateenrolled', 'dateexpiry']]
adherents['address_norm'] = adherents['address'].str.lower()
adherents['address_norm'] = adherents['address_norm'].apply(unidecode)
adherents['address_norm'] = adherents['address_norm'].str.replace( "'", " ", regex=False)
adherents['address_norm'] = adherents['address_norm'].str.replace( "\s+", " ", regex=True)
len(adherents)

31064

In [29]:
query = "SELECT * FROM statdb.iris"
adresses = pd.read_sql(query, db_conn)
adresses = adresses[['adresse', 'id_cicn2', 'irisInsee']]
adresses['address_norm'] = adresses['adresse'].str.lower()
adresses['address_norm'] = adresses['address_norm'].apply(unidecode)
adresses['address_norm'] = adresses['address_norm'].str.replace( "'", " ", regex=False)
adresses['address_norm'] = adresses['address_norm'].str.replace( "\s+", " ", regex=True)
len(adresses)

26297

# Préparation des corrections

In [30]:
# on limite aux adresses à Roubaix
rbx = adherents[adherents['zipcode'].isin(['59100'])]

# pour les personnes physiques
rbx = rbx[rbx['categorycode'].isin(['BIBL', 'CSLT', 'MEDC', 'MEDB', 'MEDA', 'CSVT'])]
len(rbx)

21940

In [34]:
rbx2 = rbx.merge(adresses, on = 'address_norm', how='left')
rbx2corr = rbx2[~rbx2['id_cicn2'].isna()]
rbx2corr = rbx2corr[(rbx2corr['state'] != rbx2corr['id_cicn2']) | (rbx2corr['altcontactcountry'] != rbx2corr['irisInsee']) ]
rbx2corr

,borrowernumber,cardnumber,userid,address,address2,city,state,zipcode,altcontactcountry,categorycode,dateenrolled,dateexpiry,address_norm,adresse,id_cicn2,irisInsee
1225,19351,X0002813615,X0002813615,55 RUE D HEM,,ROUBAIX,,59100,595120801,MEDC,2006-02-09,2026-04-17,55 rue d hem,55 RUE D HEM,24491,595120801
5088,55140,X0002646206,X0002646206,67 RUE DES FABRICANTS,N°3,ROUBAIX,,59100,595120601,MEDB,2019-07-26,2024-03-09,67 rue des fabricants,67 RUE DES FABRICANTS,8920,595120601
9593,67786,X0002574448,X0002574448,25 RUE DE L ALOUETTE,,ROUBAIX,FP016,59100,595120203,BIBL,2022-09-13,2023-09-13,25 rue de l alouette,25 RUE DE L ALOUETTE,FP016,595120601
15450,76024,X0002732954,X0002732954,16 RUE BAYARD,,ROUBAIX,,59100,595120303,BIBL,2023-11-19,2024-11-19,16 rue bayard,16 RUE BAYARD,15474,595120303
17380,78596,X0002730639,X0002730639,42 RUE HENRI DUNANT,7D1 RESIDENCE MERMOZ - chez BELOUADAH Ridha,ROUBAIX,,59100,595120603,BIBL,2024-03-21,2026-04-18,42 rue henri dunant,42 RUE HENRI DUNANT,291,595120603
17690,79051,X0002813417,X0002813417,12 RUE FENELON,,ROUBAIX,,59100,595121001,BIBL,2024-04-19,2026-04-18,12 rue fenelon,12 RUE FENELON,595120000006250,595121001
20319,82937,X0002813400,X0002813400,96 RUE DROUOT,,ROUBAIX,,59100,595120401,BIBL,2024-12-01,2025-12-01,96 rue drouot,96 RUE DROUOT,23099,595120401
21934,85285,X0002813561,X0002813561,14 RUE DE SEBASTOPOL,chambre FLORENCE,ROUBAIX,,59100,,CSLT,2025-04-18,2026-04-18,14 rue de sebastopol,14 RUE DE SEBASTOPOL,6590,595120601
21935,85288,X0002813387,X0002813387,14 RUE MAZAGRAN,,ROUBAIX,3067,59100,,CSLT,2025-04-18,2026-04-18,14 rue mazagran,14 RUE MAZAGRAN,3067,595120302
21936,85289,X0002813554,X0002813554,90 BOULEVARD DE FOURMIES,,ROUBAIX,,59100,,CSLT,2025-04-18,2026-04-18,90 boulevard de fourmies,90 BOULEVARD DE FOURMIES,20498,595120703


In [35]:
for record in rbx2corr[['cardnumber', 'userid', 'adresse', 'id_cicn2', 'irisInsee']][0:1010].to_dict(orient='records'):
    data = {
        'address': record['adresse'],
        'state': record['id_cicn2'],
        'altcontactcountry': record['irisInsee']
    }
    mod_borrower(record['userid'], data)
    print(record['cardnumber'], data)

X0002813615 {'address': '55 RUE D HEM', 'state': '24491', 'altcontactcountry': '595120801'}
X0002646206 {'address': '67 RUE DES FABRICANTS', 'state': '8920', 'altcontactcountry': '595120601'}
X0002574448 {'address': '25 RUE DE L ALOUETTE', 'state': 'FP016', 'altcontactcountry': '595120601'}
X0002732954 {'address': '16 RUE BAYARD', 'state': '15474', 'altcontactcountry': '595120303'}
X0002730639 {'address': '42 RUE HENRI DUNANT', 'state': '291', 'altcontactcountry': '595120603'}
X0002813417 {'address': '12 RUE FENELON', 'state': '595120000006250', 'altcontactcountry': '595121001'}
X0002813400 {'address': '96 RUE DROUOT', 'state': '23099', 'altcontactcountry': '595120401'}
X0002813561 {'address': '14 RUE DE SEBASTOPOL', 'state': '6590', 'altcontactcountry': '595120601'}
X0002813387 {'address': '14 RUE MAZAGRAN', 'state': '3067', 'altcontactcountry': '595120302'}
X0002813554 {'address': '90 BOULEVARD DE FOURMIES', 'state': '20498', 'altcontactcountry': '595120703'}
X0002813547 {'address': 

In [36]:
rbx_ko = rbx2[rbx2['id_cicn2'].isna()]
rbx_ko

,borrowernumber,cardnumber,userid,address,address2,city,state,zipcode,altcontactcountry,categorycode,dateenrolled,dateexpiry,address_norm,adresse,id_cicn2,irisInsee
26,379,X0001626184,X0001626184,4 A AVENUE DES COTTAGES,Bat A3 Appt C3,ROUBAIX,595120000031834,59100,595120701,BIBL,2005-03-31,2025-05-07,4 a avenue des cottages,NaN,NaN,NaN
33,471,X0001300152,X0001300152,115 C BOULEVARD DU GENERAL DE GAULLE,appt 524 rés Armenonville,ROUBAIX,19877,59100,595120701,BIBL,2005-03-31,2026-01-28,115 c boulevard du general de gaulle,NaN,NaN,NaN
72,1092,X0002580029,X0002580029,A 5 Allée Maurice Maertens,,ROUBAIX,None,59100,595120902,MEDC,2005-08-27,2025-10-08,a 5 allee maurice maertens,NaN,NaN,NaN
225,3567,X0002699493,X0002699493,127 Bld. du Gal. de Gaulle,,ROUBAIX,,59100,595120803,BIBL,2024-02-27,2025-02-27,127 bld. du gal. de gaulle,NaN,NaN,NaN
278,4178,X0002438245,X0002438245,13/42 rue Edgar Degas,None,ROUBAIX,595120000033066,59100,595121302,BIBL,2012-01-11,2025-04-11,13/42 rue edgar degas,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21838,85142,X0002815169,X0002815169,44 rue de la guinguette,,ROUBAIX,,59100,,CSLT,2025-04-08,2026-04-08,44 rue de la guinguette,NaN,NaN,NaN
21843,85148,X0002815091,X0002815091,86 RUE HENRI CARRETTE,,ROUBAIX,17793,59100,595120203,BIBL,2025-04-09,2026-04-09,86 rue henri carrette,NaN,NaN,NaN
21848,85155,X0002815022,X0002815022,5 rue J. B. Notte,Ent A Appt 42-chez ZOUGHEBI Hassiba,ROUBAIX,,59100,,BIBL,2025-04-09,2026-04-09,5 rue j. b. notte,NaN,NaN,NaN
21856,85163,X0002814681,X0002814681,118H BD DE LYON,,ROUBAIX,,59100,,BIBL,2025-04-09,2026-04-09,118h bd de lyon,NaN,NaN,NaN


In [114]:
rbx_ko.groupby('address_norm')['borrowernumber'].count().reset_index().sort_values(by='borrowernumber', ascending=False)

,address_norm,borrowernumber
45,11 rue winston churchill,19
380,46 rue du capitaine aubert,18
310,35 rue henri carrette,11
97,14 rue de l ouest,9
337,4 rue de l alma,9
...,...,...
243,26/28 rue de lille,1
245,26boulevard de la republique,1
246,27 avenue des nations unies appt 1,1
247,27 bis boulevard beaurepaire,1


In [12]:
ad = pd.read_csv("/home/kibini/kibini2/data/koha_address_corr_ok.csv")
ad = ad.merge(adresses[['id_cicn2', 'irisInsee']], on='id_cicn2', how='left')
ad

,address_norm,new_address_norm,id_cicn2,new_address,irisInsee
0,35 rue henri carrette,35 rue henri carrette,26224,35 RUE HENRI CARRETTE,595120201
1,11 b rue henri carrette,11 b rue henri carrette,19724,11 B RUE HENRI CARRETTE,595120201
2,86 rue henri carrette,86 rue henri carrette,17793,86 RUE HENRI CARRETTE,595120203
3,55 bis rue du fresnoy,55 b rue du fresnoy,25825,55 B RUE DU FRESNOY,595120102
4,54 rue henri carrette,54 rue henri carrette,1476,54 RUE HENRI CARRETTE,595120201
...,...,...,...,...,...
282,27 d14 rue la bruyere,27 d rue la bruyere,24949,27 D RUE LA BRUYERE,595120303
283,26/28 rue de lille,26 rue de lille,21594,26 RUE DE LILLE,595120601
284,26boulevard de la republique,26 boulevard de la republique,17833,26 BOULEVARD DE LA REPUBLIQUE,595120102
285,27 avenue des nations unies appt 1,27 avenue des nations unies,731,27 AVENUE DES NATIONS UNIES,595120601


In [15]:
rbx2corr = rbx[['borrowernumber', 'cardnumber', 'userid', 'address_norm']].merge(ad, on = 'address_norm', how='inner')
rbx2corr

,borrowernumber,cardnumber,userid,address_norm,new_address_norm,id_cicn2,new_address,irisInsee
0,3152,X0001495559,X0001495559,50 rue edgard quinet,50 rue edgar quinet,12733,50 RUE EDGAR QUINET,595121201
1,3764,X0002571829,X0002571829,50 boulevard du general chanzy,50 rue du general chanzy,2972,50 RUE DU GENERAL CHANZY,595120801
2,4108,X0001670905,X0001670905,65 bis rue de constantine,65 b rue de constantine,22690,65 B RUE DE CONSTANTINE,595120302
3,4917,X0002400655,X0002400655,92 rue henri carrette,92 rue henri carrette,10867,92 RUE HENRI CARRETTE,595120203
4,22379,X0002715728,X0002715728,92 rue henri carrette,92 rue henri carrette,10867,92 RUE HENRI CARRETTE,595120203
...,...,...,...,...,...,...,...,...
358,84761,X0002820156,X0002820156,34 rue saint quentin,34 rue de saint quentin,11595,34 RUE DE SAINT QUENTIN,595120903
359,84946,X0002817286,X0002817286,189 rue du cocq francais,189 rue du coq francais,2035,189 RUE DU COQ FRANCAIS,595120901
360,85070,X0002816388,X0002816388,42 rue valon,42 rue vallon,10922,42 RUE VALLON,595120203
361,85090,X0002815756,X0002815756,23 rue beaurewaert,23 rue de beaurewaert,514,23 RUE DE BEAUREWAERT,595120903


In [17]:
for record in rbx2corr[['cardnumber', 'userid', 'new_address', 'id_cicn2', 'irisInsee']][0:1010].to_dict(orient='records'):
    data = {
        'address': record['new_address'],
        'state': record['id_cicn2'],
        'altcontactcountry': record['irisInsee']
    }
    mod_borrower(record['userid'], data)
    print(record['cardnumber'], data)

X0001495559 {'address': '50 RUE EDGAR QUINET', 'state': '12733', 'altcontactcountry': '595121201'}
X0002571829 {'address': '50 RUE DU GENERAL CHANZY', 'state': '2972', 'altcontactcountry': '595120801'}
X0001670905 {'address': '65 B RUE DE CONSTANTINE', 'state': '22690', 'altcontactcountry': '595120302'}
X0002400655 {'address': '92 RUE HENRI CARRETTE', 'state': '10867', 'altcontactcountry': '595120203'}
X0002715728 {'address': '92 RUE HENRI CARRETTE', 'state': '10867', 'altcontactcountry': '595120203'}
x0002806334 {'address': '92 RUE HENRI CARRETTE', 'state': '10867', 'altcontactcountry': '595120203'}
X0001802502 {'address': '5 E RUE DE CARIHEM', 'state': '22211', 'altcontactcountry': '595121103'}
X0001844766 {'address': '4 COUR CAZIER, 68 RUE DE LA VIGNE', 'state': '31749', 'altcontactcountry': '595120402'}
X0002572796 {'address': '160 AVENUE JULES BRAME', 'state': '21820', 'altcontactcountry': '595121101'}
X0002572789 {'address': '160 AVENUE JULES BRAME', 'state': '21820', 'altcontact

X0002359700 {'address': '4 RUE DE MAUBEUGE', 'state': '12310', 'altcontactcountry': '595120802'}
X0002359816 {'address': '63 RUE JEAN BAPTISTE COROT', 'state': '20461', 'altcontactcountry': '595121201'}
X0002457130 {'address': '63 RUE JEAN BAPTISTE COROT', 'state': '20461', 'altcontactcountry': '595121201'}
X0002360959 {'address': '56 RUE PIGOUCHE', 'state': '595120000032987', 'altcontactcountry': '595120703'}
X0002427355 {'address': '0 A GALON D EAU', 'state': '20752', 'altcontactcountry': '595120403'}
X0002427706 {'address': '0 BAT A RUE DU DOCTEUR ALBERT SCHWEITZER', 'state': '311', 'altcontactcountry': '595120603'}
X0002430287 {'address': '13 RUE DU SAINT REDEMPTEUR', 'state': '9355', 'altcontactcountry': '595121001'}
X0002610108 {'address': '5 PLACEBODART TIMAL', 'state': '15886', 'altcontactcountry': '595120601'}
X0002477220 {'address': '9 RUE CHARLES PRANARD', 'state': '10918', 'altcontactcountry': '595121302'}
X0002457826 {'address': '215 QUAI DE BORDEAUX', 'state': '8263', 'al

X0002718316 {'address': '273 RUE DE LANNOY', 'state': '9033', 'altcontactcountry': '595120902'}
X0002601496 {'address': '5 RUE WINSTON CHURCHILL', 'state': '138', 'altcontactcountry': '595120603'}
X0002602493 {'address': '35 RUE HENRI CARRETTE', 'state': '26224', 'altcontactcountry': '595120201'}
X0002602509 {'address': '35 RUE HENRI CARRETTE', 'state': '26224', 'altcontactcountry': '595120201'}
X0002602516 {'address': '35 RUE HENRI CARRETTE', 'state': '26224', 'altcontactcountry': '595120201'}
X0002304830 {'address': '35 RUE HENRI CARRETTE', 'state': '26224', 'altcontactcountry': '595120201'}
X0002643892 {'address': '35 RUE HENRI CARRETTE', 'state': '26224', 'altcontactcountry': '595120201'}
X0002743394 {'address': '35 RUE HENRI CARRETTE', 'state': '26224', 'altcontactcountry': '595120201'}
X0002774336 {'address': '35 RUE HENRI CARRETTE', 'state': '26224', 'altcontactcountry': '595120201'}
X0002735740 {'address': '35 RUE HENRI CARRETTE', 'state': '26224', 'altcontactcountry': '5951202

X0002713557 {'address': '24 AVENUE JEAN JAURES', 'state': '21835', 'altcontactcountry': '595120701'}
X0002713533 {'address': '30 RUE DE LA PERCHE', 'state': '13620', 'altcontactcountry': '595120503'}
X0002220215 {'address': '16 RUE DE CROUY', 'state': '8938', 'altcontactcountry': '595120504'}
X0002769295 {'address': '12 RUE BEAUMARCHAIS', 'state': '12002', 'altcontactcountry': '595121203'}
X0002768748 {'address': '111 BOULEVARD DE MULHOUSE', 'state': '7024', 'altcontactcountry': '595121001'}
X0002768724 {'address': '54 RUE DE MONS', 'state': '11844', 'altcontactcountry': '595121001'}
X0002768670 {'address': '16 RUE DE MONS', 'state': '14309', 'altcontactcountry': '595121001'}
X0002694030 {'address': '32 RUE DE BEAUMONT', 'state': '595120000010991', 'altcontactcountry': '595120702'}
X0002692722 {'address': '81 RUE DE VALMY', 'state': '2349', 'altcontactcountry': '595120903'}
X0002692548 {'address': '2 RUE DE LA POTENNERIE', 'state': '14632', 'altcontactcountry': '595120802'}
X0002690551

X0002789736 {'address': '37 RUE EDOUARD VAILLANT', 'state': '17081', 'altcontactcountry': '595120703'}
X0002789668 {'address': '50 RUE DU GENERAL CHANZY', 'state': '2972', 'altcontactcountry': '595120801'}
X0002788234 {'address': '16 BOULEVARD DE LA REPUBLIQUE', 'state': '10894', 'altcontactcountry': '595120102'}
X0002789194 {'address': '105 RUE JACQUES PREVERT', 'state': '21256', 'altcontactcountry': '595121003'}
X0002787114 {'address': '9 ALLEE PIERRE AUGUSTE RENOIR', 'state': '20328', 'altcontactcountry': '595121301'}
X0002817903 {'address': '9 ALLEE PIERRE AUGUSTE RENOIR', 'state': '20328', 'altcontactcountry': '595121301'}
X0002787022 {'address': '43 RUE COLIGNY', 'state': '19403', 'altcontactcountry': '595120801'}
X0002794730 {'address': '90 RUE DROUOT', 'state': '10445', 'altcontactcountry': '595120401'}
X0002794693 {'address': '18 RUE DE SAINT AMAND', 'state': '13898', 'altcontactcountry': '595120903'}
X0002794471 {'address': '18 PLACE DES TROIS PONTS', 'state': '59512000000672

In [40]:
rbx.groupby(["state", "address"])['borrowernumber'].count().reset_index().to_csv("med_adresses_20250418.csv", index=False)